In [1]:
from copy import deepcopy as dcopy
import matplotlib 
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import rc
%matplotlib inline
rc('text', usetex=True)
font = {'weight' : 'bold','size'   : 22}
matplotlib.rc('font', **font)
#############################################
import warnings
warnings.filterwarnings("ignore")
#############################################
import os,sys
import numpy as np
import healpy as hp
import healpy.newvisufunc as hpn
import astropy.io.fits as fits
#import pandas as pd
import pymaster as nmt
#############################################
sys.path.insert(1, '/media/BINGODATA1/ComponentSeparation/beam_analyzes/scripts')
import handling_data          as hdata
#############################################

In [4]:
path        = "/media/BINGODATA1/ComponentSeparation/MAPS/HI"
idir        = "flask_maps_512"
output_info = {"field":"HI", 
               "frequency":{"min":980,"max":1260,'nbands':30},
               "coverage":'full',
               'beam': {'model':None, 'fwhm':10}, 
               'stokes':'I',
               "freq_unit":"MHz", "stokes_unit":"mk", "fwhm_unit":"arcmin",
               "realization":['0100','0010'],
               "output_dir":'/media/BINGODATA1/ComponentSeparation/building_dataset/dataset/HI512',
               "clear_output_dir":True}

# Detailed Description - How to use

### Variables used

- path: path to directory where the one containing all realizations is
- idir: the name os the directory with all realization FITS files

- field: type of field. For instance: HI, FG, N, WN, ...
    - FG: each kind of astrophysical component should have the short name composing as follow:
        - [AME]: Anomalous Microwave Background
        - [CIB]: Cosmic Infrared Background
        - [CMB]: Cosmic Microwave Background
        - [SYN] : Synchrotron
        - [FRP]: Faint Radio Point sources
        - [FF] :  Free-Free emission
        - [SZ]: Sunyaev-Zeldovich emission
    
    - HI:  neutral hydrogen
    
    - N: Noise could be
        - [uWN]: uncorrelated White Noise
        - [cWN]: correlated White Noise
        - [1F]:  pink Noise or 1/f-noise
        - [PLK]: Polarization LeaKage
        - others
        PS.: if the component is a composition of many FG is necessary use onlu FG and specify on the FITS head each component
    
    - PRIOR: We will keep the same nomeclature come from the previous works and assume PRIOR as a composition of HI + N, where N could be uWN, cWN or other kind of 
        - PS.: it should be specified on the FITS head each component
    
    - INPUT or SKY: It'll be assumed as a composition of sky emissions + instrumental noises.
    
- S: Stokes parameters: It should be speficied which type of Stokes parameter the file represent 
    - I: intensity
    - Q: Q-pol parameter
    - U: U-pol parameter
    - W: W-pol parameter
    - P: P-pol parameter
    - p: p-pol parameter
    

- frequency: Informations related to frequancy properties, as minimum and maximum,as well as number of bands used    

- coverage: Type of covering: full or partial. If the map(s) is(are) masked then it's necessary to use "partial". Otherwise, if all sky, "full".

- beam: Specification of the beam. If there is no beam convolution, it should be assumed the type NONE. If there is a convolution, first you need to include the full-width-at-half-maximum resolution in arcmin. Then, the type of beam model. Acronynms of the beam model are as follow:
    - [ZN] : Zernike Polynomial
    - [G] : Gaussian
    - [JC] : Spherical Bessel
    - [CO] : Cosine Polynomial
    - [LA] : Laguerre
    - [HM] : Hermite
    - [AI] : Airy 

- realization: the ID identification of the realization (for instance, '0001', '0210', ...). it could be put in a array (or list) or alone way. If you want to use more than one, you will need to use as a array (or list). If you want to use all realizations on the 'pathdir' directory, please, wheather use None or the string "all".

- output_dir: the complete path where will be saved all multimaps. If type None, it'll be saved in a standard directory (to be created or not)

- clear_output_dir: True: If the directory already exists, it'll be cleaned

In [5]:
pathdir   = hdata.os.path.join(path,idir) #<-- path to diretories with all maps
filenames = hdata.get_allfilenames(dirpath_=pathdir) #all filenames in the directory
_ids_     = hdata.get_all_realizations_id(filenames) #all realization names
idic      = hdata.get_filenames4realizations(filenames_=filenames, ids_=_ids_) #all filenames for a specific realization
###########
ifile     = np.random.choice(idic[output_info['realization'][0]]) #Choice a specific realization name just to test
ginfo     = hdata.get_info_from_filename(dirname_= idir, filename_=ifile, 
                                         numin_ =output_info['frequency']['min'], 
                                         numax_ =output_info['frequency']['max'], 
                                         nbands_=output_info['frequency']['nbands']) #info from filenames
new_name  = hdata.return_new_FITSfilename(ginfo, output_info, add_info=None) #new FITS name for that map cube
###########
vec_hdu0  = ginfo['frequency']['nu']
hdr_hdu0  = hdata.creating_primary_FITSheader(output_info)
vec_hdu1  = hdata.building_cubemaps_from_realization(pathdir_=pathdir, filenames_=idic[output_info['realization'][0]]) #extracting all maps for a specific realization and building a cube
hdr_hdu1  = hdata.creating_multimap_FITSheader(new_name, 
                                               freq_unit  =output_info["freq_unit"], 
                                               stokes_unit=output_info["stokes_unit"], 
                                               fwhm_unit  =output_info["fwhm_unit"]) #Header to be used 

In [6]:
print("Each name of INPUT map looks like: {}".format(ifile) )
print("Multimaps new name: {}".format(new_name) )

Each name of INPUT map looks like: HImap-r0100-f1z26.fits
Multimaps new name: HI_I_512_980mhz1260mhz_30bins_full_L0100.fits


In [7]:
try:
    os.remove(new_name)
except: 
    pass
new_name = "".join(("test_",new_name)) #I'll change the 'new_name' just to show how this work. In another case, keep the same new_name, that's, doesn't change it
hdu0 = fits.PrimaryHDU(header=hdr_hdu0,data=vec_hdu0)
hdu1 = fits.ImageHDU(  header=hdr_hdu1,data=vec_hdu1, name="MULTIMAPS")
hdul = fits.HDUList([hdu0,hdu1])
hdul.writeto(new_name,overwrite=True)
hdul.close()

In [8]:
#General info
hdul.info()

Filename: (No file associated with this HDUList)
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       9   (31,)   float64   
  1  MULTIMAPS     1 ImageHDU        22   (3145728, 30)   float64   


In [9]:
#PRIMARY HDU -- HDU 0
#
# Here you'll see informations about redshift vector containing all intervals of redshifts
# Minimal and maximal frequencies used for and number of bands (or bins, which is the same here)
print('HEADER:\n')
print(repr(hdul[0].header))

HEADER:

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -64 / array data type                                
NAXIS   =                    1 / number of array dimensions                     
NAXIS1  =                   31                                                  
EXTEND  =                    T                                                  
FREQ_MIN=                  980                                                  
FREQ_MAX=                 1260                                                  
NBANDS  =                   30                                                  
COMMENT PRIMARY HDU CONTAIN THE REDSHIFT VECTOR                                 


In [13]:
hdul[0].data.size

31

In [10]:
print('DATA:\n')
print(repr(hdul[0].data))

DATA:

array([ 980.  ,  989.33,  998.67, 1008.  , 1017.33, 1026.67, 1036.  ,
       1045.33, 1054.67, 1064.  , 1073.33, 1082.67, 1092.  , 1101.33,
       1110.67, 1120.  , 1129.33, 1138.67, 1148.  , 1157.33, 1166.67,
       1176.  , 1185.33, 1194.67, 1204.  , 1213.33, 1222.67, 1232.  ,
       1241.33, 1250.67, 1260.  ])


In [27]:
#IMAGE HDU -- HDU 1 -- ALL MAPS IN A MATRIX REPRESENTATION
#
# HERE you can look at all information describing the multimaps (all maps in matrix representation)
# You can see which is the field, Stokes parameter, NSIDE HealPIX resolution, Project covering,
# If it's assumed a beam convolution or not, and if so which are FHWM and the Beam model assumed
print('HEADER:\n')
print(repr(hdul[1].header))

HEADER:

XTENSION= 'IMAGE   '           / Image extension                                
BITPIX  =                  -64 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =              3145728                                                  
NAXIS2  =                   30                                                  
PCOUNT  =                    0 / number of parameters                           
GCOUNT  =                    1 / number of groups                               
FIELD   = 'HI      '                                                            
STOKES  = 'I       '           / mk                                             
NSIDE   =                  512                                                  
FREQ_MIN=                980.0 / in MHz                                         
FREQ_MAX=               1260.0 / in MHz                                         
NBINS   =          

In [28]:
print('DATA:\n')
print(repr(hdul[1].data))

DATA:

array([[ 5.47993556e-02,  5.26039779e-01,  7.92899355e-02, ...,
         2.64430624e-02,  5.89685934e-03,  1.58316139e-02],
       [ 4.22787368e-02,  3.91779318e-02,  1.26857944e-02, ...,
        -2.82995519e-03,  2.04597483e-04,  2.65518706e-02],
       [-8.49895645e-03, -8.58006906e-03, -8.79507512e-03, ...,
         9.74427350e-03,  6.45814696e-03,  4.93700840e-02],
       ...,
       [ 6.39779419e-02,  2.39880243e-03,  4.90189530e-02, ...,
         2.07588315e-01,  9.86814350e-02,  1.12318106e-01],
       [ 2.07764413e-02,  1.73066240e-02,  3.31539735e-02, ...,
         4.55153510e-02,  7.51858950e-02, -9.54377232e-04],
       [-3.27165029e-03,  5.37990639e-03,  1.05088554e-01, ...,
         4.75355983e-03,  5.73629960e-02,  2.23025028e-02]])


In [29]:
print("Size of the FITS file: {}".format(os.popen("du -sh {}".format(new_name)).readlines()[0].split()[0]))

Size of the FITS file: 721M


In [30]:
iname = new_name
with fits.open(os.path.join(os.getcwd(),"",iname)) as h:
    hdul = dcopy(h)
hdul.info()

Filename: (No file associated with this HDUList)
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       9   (31,)   float64   
  1  MULTIMAPS     1 ImageHDU        22   (3145728, 30)   float64   


### MASK